# Get System Metadata

[System metadata](https://dataone-architecture-documentation.readthedocs.io/en/latest/apis/Types.html#Types.SystemMetadata) is low level metadata associated with every object synchronized with DataONE.

Retrieving the system metadata for an object is accomplished by the [`getSystemMetadata(id)`](https://dataone-architecture-documentation.readthedocs.io/en/latest/apis/MN_APIs.html#MNRead.getSystemMetadata) call which accepts a single parameter, the identifier of the object to retrieve.


In [1]:
# include some utility data and methods
import notebook_utils as nbu

# Import the library and create a client instance
from d1_client import baseclient_2_0

cn_base_url = "https://cn.dataone.org/cn"
client = baseclient_2_0.DataONEBaseClient_2_0(cn_base_url)

identifier = "urn:uuid:f310f0cb-1f3d-464c-9274-5036fb8eb471"

sysmeta = client.getSystemMetadata(identifier)

# System metadata object is returned
print(sysmeta)

# Show the xml representation of the system metadata
print(nbu.asXml(sysmeta, max_lines=-1))

<?xml version="1.0" ?>
<ns1:systemMetadata xmlns:ns1="http://ns.dataone.org/service/types/v2.0">
  <serialVersion>1</serialVersion>
  <identifier>urn:uuid:f310f0cb-1f3d-464c-9274-5036fb8eb471</identifier>
  <formatId>https://eml.ecoinformatics.org/eml-2.2.0</formatId>
  <size>49777</size>
  <checksum algorithm="MD5">1106c26cc194813a64b1549cb0ca8093</checksum>
  <submitter>http://orcid.org/0000-0001-6544-2035</submitter>
  <rightsHolder>http://orcid.org/0000-0001-6544-2035</rightsHolder>
  <accessPolicy>
    <allow>
      <subject>CN=knb-data-admins,DC=dataone,DC=org</subject>
      <permission>read</permission>
      <permission>write</permission>
      <permission>changePermission</permission>
    </allow>
    <allow>
      <subject>public</subject>
      <permission>read</permission>
    </allow>
  </accessPolicy>
  <replicationPolicy replicationAllowed="false"/>
  <obsoletes>urn:uuid:25332fbe-8b05-4c72-852f-c3ee3021da2f</obsoletes>
  <archived>false</archived>
  <dateUploaded>2020-0

## Accessing properties of the system metadata

Properties of the system metadata can be accessed directly from the system metadata instance which is a [PyXB](https://pypi.org/project/PyXB/) object resulting from the deserialization of the XML message. Note some properties are complex (e.g. `identifier`) that are accessed through a `.value()` accessor. For example:


In [4]:
print(f"Identifier: {sysmeta.identifier.value()}")
print(f"Series Identifier: {nbu.propertyStr(sysmeta.seriesId)}")
print(f"Modified: {sysmeta.dateSysMetadataModified}")
print(f"Uploaded: {sysmeta.dateUploaded}")
print(f"Format ID: {sysmeta.formatId}")
print(f"Size: {sysmeta.size}")
print(f"Checksum: hash://{sysmeta.checksum.algorithm.lower()}/{sysmeta.checksum.value()}")
print(f"Origin Member Node: {nbu.propertyStr(sysmeta.originMemberNode)}")
print(f"Authoritative Member Node: {nbu.propertyStr(sysmeta.authoritativeMemberNode)}")
print(f"Obsoletes: {nbu.propertyStr(sysmeta.obsoletes)}")
print(f"Obsoleted By: {nbu.propertyStr(sysmeta.obsoletedBy)}")
print("Access policy rules:")
for rule in sysmeta.accessPolicy.allow:
    print(f"  {', '.join(map(lambda S: S.value(), rule.subject))}  can  {', '.join(rule.permission)}")
print(f"Replication allowed: {sysmeta.replicationPolicy.replicationAllowed}")
print("Replicas of this object:")
for replica in sysmeta.replica:
    print(f"  {replica.replicaMemberNode.value():15} {replica.replicationStatus:10} {replica.replicaVerified}")


Identifier: urn:uuid:f310f0cb-1f3d-464c-9274-5036fb8eb471
Series Identifier: 
Modified: 2020-08-07 04:36:02.485000+00:00
Uploaded: 2020-08-07 04:36:01.550000+00:00
Format ID: https://eml.ecoinformatics.org/eml-2.2.0
Size: 49777
Checksum: hash://md5/1106c26cc194813a64b1549cb0ca8093
Origin Member Node: urn:node:KNB
Authoritative Member Node: urn:node:KNB
Obsoletes: urn:uuid:25332fbe-8b05-4c72-852f-c3ee3021da2f
Obsoleted By: 
Access policy rules:
  CN=knb-data-admins,DC=dataone,DC=org  can  read, write, changePermission
  public  can  read
Replication allowed: false
Replicas of this object:
  urn:node:KNB    completed  2020-08-07 12:48:55.328000+00:00
  urn:node:CN     completed  2020-08-07 12:48:55.368000+00:00


## Error Handling

Errors resulting from a call to a DataONE API will normally result in a `DataONEException` which can include detail about the cause of the error.

In [3]:
# Try and retrieve non-existing system metadata
import d1_common.types.exceptions

identifier = "something_bogus"
try:
    sysmeta = client.getSystemMetadata(identifier)
except d1_common.types.exceptions.DataONEException as e:
    print(e)

name: NotFound
errorCode: 404
detailCode: 1420
description: No system metadata could be found for given PID: something_bogus
traceInformation:
  < Accept-Encoding: gzip, deflate
  < Accept: */*
  < Charset: utf-8
  < Connection: keep-alive
  < GET /cn/v2/meta/something_bogus HTTP/1.1
  < Host: cn.dataone.org
  < User-Agent: DataONE-Python/3.5.0 +http://dataone.org/
  
  > Access-Control-Allow-Credentials: true
  > Access-Control-Allow-Headers: Authorization, Content-Type, Location, Content-Length, x-annotator-auth-token, Cache-Control
  > Access-Control-Allow-Methods: POST, GET, OPTIONS, PUT, DELETE
  > Access-Control-Allow-Origin: 
  > Access-Control-Expose-Headers: Content-Length, Content-Type, Location
  > Connection: Keep-Alive
  > Content-Length: 201
  > Content-Type: text/xml
  > Date: Tue, 11 Aug 2020 12:42:26 GMT
  > HTTP/1.1 404 404
  > Keep-Alive: timeout=5, max=100
  > Server: Apache/2.4.43 (Ubuntu)
  > Set-Cookie: JSESSIONID=6277636C70DDAE65E22AD9F30C3E8681; Path=/metacat; 